# 중간중간 빠진 부분은 수업시간에 뺀다고 했던 내용입니다.

# 고로 나와있는 코드들만 범위임!!

# Data Loading, Storage,

In [11]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress = True)

# 텍스트 파일에서 데이터를 읽고 쓰는 법

In [12]:
!cat examples/ex1.csv




9,10,11,12,foo

In [13]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


read_table에 구분자를 쉼표로 지정해서 읽어올 수도 있다.

In [14]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
!cat exapmles/ex2.csv

cat: exapmles/ex2.csv: No such file or directory


pandas가 자동으로 컬럼 이름을 생성하도록 하거나 우리가 직접 컬럼 이름을 지정한다.

In [16]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [17]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


message 컬럼을 색인으로 하는 DataFrame을 반환하려면 index_col 인자에 4번째 컬럼 또는 'message' 이름을 가진 컬럼을 지정해서 색인으로 만들 수 있다.

In [18]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


계층적 색인을 지정하고 싶다면 컬럼 번호나 이름의 리스트를 넘기면 된다.

In [19]:
!cat examples/csv_mindex.csv
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [20]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

이 파일은 필드가 여러 개의 공백 문자로 구분되어 있으므로 이를 표현할 수 있는 정규 표현식 \s+를 사용해서 처리할 수도 있다. 

In [21]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


이 경우 첫번째 로우는 다른 로우보다 컬럼이 하나 적기 때문에 read_table은 첫번째 컬럼이 DataFrame의 색인이 되어야한다고 추론한다.

파서 함수는 파일 형식에서 발생할 수 있는 매우 다양한 예외를 잘 처리할 수 있도록 많은 추가 인자를 가지고 있는데, 예를 들면 skiprows를 이용해서 첫번째, 세번째, 네번째 로우를 건너뛸 수 있다.

In [24]:
!cat examples/ex4.csv
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])







9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


기본적으로 pandas는 NA나 NULL처럼 흔히 통용되는 문자를 비어 있는 값으로 사용한다.

In [13]:
!cat examples/ex5.csv




three,9,10,11,12,foo

In [26]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [27]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


na_values 옵션은 리스트나 문자열 집합을 받아서 누락된 값을 처리한다.

In [30]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


컬럼마다 다른 NA 문자를 사전값으로 넘겨서 처리할 수도 있다.

In [31]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


# 텍스트 파일 조금씩 읽어오기

In [32]:
pd.options.display.max_rows = 10

In [33]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


파일 전체를 읽는 대신 처음 몇 줄만 읽어보고 싶다면 nrow 옵션을 주면 된다.

In [43]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


파일을 여러 조각으로 나누어서 읽고 싶다면 chunksize 옵션으로 로우의 개수를 주면 된다.

In [35]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

read_csv에서 반환된 TextParser 객체를 이용해서 chunksize에 따라 분리된 파일들을 순회할 수 있다.

In [40]:
tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot

0    202.0
1    182.0
2    197.0
3    240.0
4    234.0
     ...  
V    397.0
W    389.0
X    448.0
Y    428.0
Z    381.0
Length: 36, dtype: float64

In [42]:
tot = tot.sort_values(ascending=False)
tot

E    473.0
M    461.0
K    460.0
X    448.0
U    443.0
     ...  
5    202.0
0    202.0
2    197.0
9    195.0
1    182.0
Length: 36, dtype: float64

In [37]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)
tot[:10]

/Users/otaehun/opt/anaconda3/envs/dhxogns920/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

TextParser는 임의 크기의 조각을 읽을 수 있는 get_chunk 메서드도 포함하고 있다..

In [45]:
result["key"].value_counts()

E    368
X    364
L    346
O    343
Q    340
    ... 
5    157
2    152
0    151
9    150
1    146
Name: key, Length: 36, dtype: int64

In [46]:
piece['key'].value_counts()

K    42
M    41
U    39
Y    38
P    38
     ..
9    15
8    15
5    15
7    14
1    12
Name: key, Length: 36, dtype: int64

In [47]:
piece

,one,two,three,four,key
9000,0.467976,-0.038649,-0.295344,-1.824726,B
9001,-0.358893,1.404453,0.704965,-0.200638,M
9002,-0.501840,0.659254,-0.421691,-0.057688,N
9003,0.204886,1.074134,1.388361,-0.982404,N
9004,0.354628,-0.133116,0.283763,-0.837063,Y
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [48]:
tot

E    473.0
M    461.0
K    460.0
X    448.0
U    443.0
     ...  
5    202.0
0    202.0
2    197.0
9    195.0
1    182.0
Length: 36, dtype: float64

# 데이터를 텍스트 형식으로 기록하기

In [49]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


DataFrame의 to_csv 메서드를 이용하면 데이터를 쉼표로 구분된 형식으로 파일에 쓸 수 있다.

In [50]:
data.to_csv('examples/out.csv')
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


물론 다른 구분자도 사용가능

In [51]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


결과에서 누락된 값은 비어 있는 문자열로 나타나는데, 이것 역시 원하는 값으로 지정 가능하다.

In [30]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


다른 옵션을 명시하지 않으면 로우와 컬럼 이름이 기록된다.

로우와 컬럼 이름을 포함하지 않으려면 다음과 같이 한다.

In [52]:
data.to_csv(sys.stdout)

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [31]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


컬럼의 일부분만 기록할 수도 있으며, 순서를 직접 지정할 수도 있다.

In [53]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series에도 to_csv 메서드가 존재한다.

In [54]:
dates = pd.date_range('6/11/20', periods=7)
dates

DatetimeIndex(['2020-06-11', '2020-06-12', '2020-06-13', '2020-06-14',
               '2020-06-15', '2020-06-16', '2020-06-17'],
              dtype='datetime64[ns]', freq='D')

In [55]:
ts = pd.Series(np.arange(7), index=dates)
ts

2020-06-11    0
2020-06-12    1
2020-06-13    2
2020-06-14    3
2020-06-15    4
2020-06-16    5
2020-06-17    6
Freq: D, dtype: int64

In [56]:
ts.to_csv('examples/tseries.csv')

In [57]:
!cat examples/tseries.csv

,0
2020-06-11,0
2020-06-12,1
2020-06-13,2
2020-06-14,3
2020-06-15,4
2020-06-16,5
2020-06-17,6


# JSON 데이터

JSON은 CSV 같은 표 형식의 텍스트보다 좀 더 유연한 데이터 형식이다.

In [58]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [59]:
import json

JSON 문자열을 파이썬 형태로 변환하기 위해서는 json.loads를 사용한다.

In [61]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

json.dumps는 파이썬 객체를 JSON 형태로 변환한다.

In [62]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

JSON 객체의 리스트를 사전을 담고 있는 리스트로 변환하여 DataFrame 생성자로 넘기고 데이터 필드를 선택할 수 있다.

In [63]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


pandas.read_json은 자동으로 JSON 데이터셋을 Series나 DataFrame으로 변환할 수 있다.

In [66]:
!cat examples/example.json

별다른 옵션이 주어지지 않았을 경우 pandas.read_json은 JSON 배열에 담긴 각 객체를 테이블의 로우로 간주한다.

In [67]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


pandas의 데이터를 JSON으로 저장하는 한가지 방법은 Series나 DataFrame의 to_json 함수를 이용하는 것이다.

In [68]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [45]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


# 이진 데이터 형식

데이터를 효율적으로 저장하는 가장 손쉬운 방법은 파이썬에 기본으로 내장되어 있는 pickle 직렬화를 사용해 데이터를 이진 형식으로 저장하는 것

pandas 객체는 모두 pickle을 이용해서 데이터를 저장하는 to_pickle 메서드를 가지고 있다.

In [69]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [71]:
frame.to_pickle('examples/frame_pickle')

pickle로 직렬화된 객체는 내장 함수인 pickle로 직접 불러오거나 아니면 좀 더 편리한 pickle 함수인 pandas.read_pickle 메서드를 이용하여 불러올 수 있다.

In [72]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* Bcolz : Blocs 압축 알고리즘에 기반한 압축이 가능한 컬럼지향 바이너리 포맷

* Feaeat : Feather는 아파치 에로우의 메모리 포맷을 사용

# HDF5 형식 사용하기

In [49]:
frame = pd.DataFrame({'a': np.random.randn(100)})
frame

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


# -------------------------------------------------------------------------

## 코드 실행이  안되는데  왜 일까...?

In [52]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [ ]:
store['obj1']

In [ ]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [ ]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

In [ ]:
os.remove('mydata.h5')

# 마이크로소프트 엑셀 파일에서 데이터 읽어오기

ExcelFile 클래스를 사용하려면 xls나 xlsx 파일의 경로를 지정하여 객체를 생성해야한다.

In [73]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
xlsx

시트에 있는 데이터는 parse 함수를 이용해서 DataFrame으로 읽어올 수 있다.

In [74]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


한 파일에서 여러 시트를 읽어오려면 Excelfile을 생성하면 빠르지만 간단하게는 pandas, read_excel에 파일 이름만 넘겨도 된다.

In [75]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


pandas 데이터를 엑셀 파일로 저장하고 싶다면 ExcelWrite를 생성해서 데이터를 기록하고 pandas 객체의 to_excel 메서드로 넘기면 된다

In [76]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
writer

In [77]:
frame.to_excel(writer, 'Sheet1')
writer.save()

ExcelWriter를 사용하지 않고 to_excel 메서드에 파일 경로만 넘겨도 된다.

In [80]:
frame.to_excel('examples/writer.xlsx')

In [82]:
test = pd.read_excel('examples/writer.xlsx')
test

,Unnamed: 0,Unnamed: 0.1,a,b,c,d,message
0,0,0,1,2,3,4,hello
1,1,1,5,6,7,8,world
2,2,2,9,10,11,12,foo


# 웹 API와 함께 사용하기

pandas 깃허브에서 최근 30개의 이슈를 가져오려면 requests 라이브러리를 이용해서 다음과 같은 GET HTTP 요청을 생성하면 된다.

In [83]:
import requests

In [84]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [85]:
data = resp.json()
data[0]['title']

'ENH: implement Timestamp _as_reso, _as_unit'

In [86]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,47333,"ENH: implement Timestamp _as_reso, _as_unit","[{'id': 3713792788, 'node_id': 'LA_kwDOAA0YD87...",open
1,47332,TST: tzdata is a required dependency when testing,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
2,47331,BUG: concat not sorting mixed column names whe...,"[{'id': 13098779, 'node_id': 'MDU6TGFiZWwxMzA5...",open
3,47330,BUG: assert_index_equal ignoring names when ch...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,47329,BUG: concat losing column dtype for extension ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
...,...,...,...,...
25,47304,TYP: Series.quantile,[],open
26,47301,BUG: uint64 series resampling,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
27,47298,TYP: NumericDtype._standardize_dtype,[],open
28,47294,BUG: uint16 inserted as int16 when assigning r...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
